In [3]:
import os
from pathlib import Path

import google.generativeai as genai
from PIL import Image
from dotenv import load_dotenv

load_dotenv()

os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

generation_config = {
    "temperature": 0.1,
    "top_p": 0.90,
    "max_output_tokens": 2048,
}

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
]
model = genai.GenerativeModel(model_name="gemini-1.0-pro-vision-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)


In [4]:
def get_gemini_repsonse(prompt, original_image):
    model = genai.GenerativeModel(model_name="gemini-1.0-pro-vision-latest",
                                  generation_config=generation_config,
                                  safety_settings=safety_settings)

    response = model.generate_content([prompt, original_image])
    return response.text

In [ ]:

def takemura(filenames, uploaded_file, product_details):

            webp_path = os.path.splitext(img_path)[0] + '.webp'
            img.save(webp_path, 'webp')

    # Now your images are in webp format and you can read them into your program
    original_image_parts = [{"mime_type": "image/webp",
                             "data": uploaded_file.read()}]  # Use read method instead of read_bytes

    # Now your images are in webp format and you can read them into your program
    image_parts = [{"mime_type": "image/webp",
                    "data": Path(f"{os.path.splitext(os.path.join(directory, f))[0]}.webp").read_bytes()} for f in
                   filenames if os.path.isfile(f"{os.path.splitext(os.path.join(directory, f))[0]}.webp")]

    prompt_parts = [original_image_parts[0],
                    f""",
        I want you to ınspect the file above and compare it with product images with below. While doing that, please follow these steps:
            1- For ranking similarity, the order of criteria importance should be orderly {product_details}. If the first criteria is not enough to rank, then the second criteria should be used and so on.
            2- While giving output, give the original filenames please cause i don't understand when you say \"first image, second image\" etc.
            3- Results should only have direct filenames from {filenames} which similar 5 product image to original image like text below:
                    >>(image_name).jpg
                    >>(image_name).jpg.
                    .....
            4- The results should only include the top 5 products that are most similar to the original product image from the provided list of product images.",
        """
                    ]

    for i in range(1, 6):
        prompt_parts.append(image_parts[i])
        prompt_parts.append(" ")

    response = model.generate_content(prompt_parts)
    print(response.text)


    cleaned_response = "\n".join(line.lstrip('>>').strip() if line.startswith('>>') else line for line in response.text.split('\n'))
    cleaned_response = "\n".join(line[-22:] for line in cleaned_response.split('\n'))

    return cleaned_response
